In [1]:
import mindspore
from mindspore.dataset import GeneratorDataset, transforms

from mindnlp.engine import Trainer

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/lvyufeng/miniconda3/envs

In [2]:
# prepare dataset
class SentimentDataset:
    """Sentiment Dataset"""

    def __init__(self, path):
        self.path = path
        self._labels, self._text_a = [], []
        self._load()

    def _load(self):
        with open(self.path, "r", encoding="utf-8") as f:
            dataset = f.read()
        lines = dataset.split("\n")
        for line in lines[1:-1]:
            label, text_a = line.split("\t")
            self._labels.append(int(label))
            self._text_a.append(text_a)

    def __getitem__(self, index):
        return self._labels[index], self._text_a[index]

    def __len__(self):
        return len(self._labels)

In [3]:
# download dataset
!wget https://baidu-nlp.bj.bcebos.com/emotion_detection-dataset-1.0.0.tar.gz -O emotion_detection.tar.gz
!tar xvf emotion_detection.tar.gz

--2024-11-22 16:04:29--  https://baidu-nlp.bj.bcebos.com/emotion_detection-dataset-1.0.0.tar.gz
Resolving baidu-nlp.bj.bcebos.com (baidu-nlp.bj.bcebos.com)... 198.18.0.62
Connecting to baidu-nlp.bj.bcebos.com (baidu-nlp.bj.bcebos.com)|198.18.0.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1710581 (1.6M) [application/x-gzip]
Saving to: ‘emotion_detection.tar.gz’

emotion_detection.t 100%[===================>]   1.63M  6.97MB/s    in 0.2s    

2024-11-22 16:04:30 (6.97 MB/s) - ‘emotion_detection.tar.gz’ saved [1710581/1710581]

data/
data/test.tsv
data/infer.tsv
data/dev.tsv
data/train.tsv
data/vocab.txt


In [4]:
def process_dataset(source, tokenizer, max_seq_len=64, batch_size=32, shuffle=True):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'

    column_names = ["label", "text_a"]
    
    dataset = GeneratorDataset(source, column_names=column_names, shuffle=shuffle)
    # transforms
    type_cast_op = transforms.TypeCast(mindspore.int32)
    def tokenize_and_pad(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text)
        return tokenized['input_ids'], tokenized['attention_mask']
    # map dataset
    dataset = dataset.map(operations=tokenize_and_pad, input_columns="text_a", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=[type_cast_op], input_columns="label", output_columns='labels')
    # # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})

    return dataset

昇腾NPU环境下暂不支持动态Shape，数据预处理部分采用静态Shape处理：

In [5]:
from mindnlp.transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.11/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [6]:
tokenizer.pad_token_id

0

In [7]:
dataset_train = process_dataset(SentimentDataset("data/train.tsv"), tokenizer)
dataset_val = process_dataset(SentimentDataset("data/dev.tsv"), tokenizer)
dataset_test = process_dataset(SentimentDataset("data/test.tsv"), tokenizer, shuffle=False)

In [8]:
dataset_train.get_col_names()

['input_ids', 'attention_mask', 'labels']

In [9]:
type(dataset_train)

mindspore.dataset.engine.datasets.BatchDataset

In [10]:
print(next(dataset_train.create_dict_iterator()))

{'input_ids': Tensor(shape=[32, 64], dtype=Int64, value=
[[ 101, 6612, 2209 ...    0,    0,    0],
 [ 101, 2769, 7270 ...    0,    0,    0],
 [ 101, 5439, 2038 ...    0,    0,    0],
 ...
 [ 101, 6929, 2769 ...    0,    0,    0],
 [ 101, 2769, 6230 ...    0,    0,    0],
 [ 101, 2218, 5464 ...    0,    0,    0]]), 'attention_mask': Tensor(shape=[32, 64], dtype=Int64, value=
[[1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 ...
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0]]), 'labels': Tensor(shape=[32], dtype=Int32, value= [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 
 1, 1, 1, 1, 1, 1, 2, 1])}


In [11]:
from mindnlp.transformers import BertForSequenceClassification, BertModel

# set bert config and define parameters for training
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=3)

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from mindnlp.engine import TrainingArguments

training_args = TrainingArguments(
    output_dir="bert_emotect_finetune",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    num_train_epochs=3.0
)

In [13]:
from mindnlp import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics
)

In [15]:
# start training
trainer.train()

  0%|                                                                   | 1/906 [00:12<3:07:56, 12.46s/it]

-\|/-\|/

 33%|██████████████████████▎                                            | 302/906 [03:52<04:28,  2.25it/s]

{'loss': 0.3296, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}
-


 88%|████████████████████████████████████████████████████████████▉        | 30/34 [00:02<00:00, 21.85it/s]
                                                                                                          
100%|█████████████████████████████████████████████████████████████████████| 34/34 [00:02<00:00, 22.53it/s]
                                                                                                          

...


 91%|████████████████████████████████████████████████████████████████████████       | 31/34 [00:01<00:00, 27.72it/s]
                                                                                                                    
100%|███████████████████████████████████████████████████████████████████████████████| 34/34 [00:01<00:00, 27.80it/s]
                                                                                                                    

{'eval_loss': 0.06156477704644203, 'eval_accuracy': 0.9814814814814815, 'eval_runtime': 1.7696, 'eval_samples_per_second': 19.214, 'eval_steps_per_second': 2.826, 'epoch': 3.0}


100%|█████████████████████████████████████████████████████████████████████████████| 906/906 [09:08<00:00,  1.65it/s]

{'train_runtime': 548.775, 'train_samples_per_second': 52.83, 'train_steps_per_second': 1.651, 'train_loss': 0.215978719277624, 'epoch': 3.0}


TrainOutput(global_step=906, training_loss=0.215978719277624, metrics={'train_runtime': 548.775, 'train_samples_per_second': 52.83, 'train_steps_per_second': 1.651, 'train_loss': 0.215978719277624, 'epoch': 3.0})

In [16]:
dataset_infer = SentimentDataset("data/infer.tsv")

In [17]:
def predict(text, label=None):
    label_map = {0: "消极", 1: "中性", 2: "积极"}

    text_tokenized = Tensor([tokenizer(text).input_ids])
    logits = model(text_tokenized)
    predict_label = logits[0].asnumpy().argmax()
    info = f"inputs: '{text}', predict: '{label_map[predict_label]}'"
    if label is not None:
        info += f" , label: '{label_map[label]}'"
    print(info)

In [18]:
from mindspore import Tensor

for label, text in dataset_infer:
    predict(text, label)

inputs: '我 要 客观', predict: '中性' , label: '中性'
inputs: '靠 你 真是 说 废话 吗', predict: '消极' , label: '消极'
inputs: '口嗅 会', predict: '中性' , label: '中性'
inputs: '每次 是 表妹 带 窝 飞 因为 窝路痴', predict: '中性' , label: '中性'
inputs: '别说 废话 我 问 你 个 问题', predict: '消极' , label: '消极'
inputs: '4967 是 新加坡 那 家 银行', predict: '中性' , label: '中性'
inputs: '是 我 喜欢 兔子', predict: '积极' , label: '积极'
inputs: '你 写 过 黄山 奇石 吗', predict: '中性' , label: '中性'
inputs: '一个一个 慢慢来', predict: '中性' , label: '中性'
inputs: '我 玩 过 这个 一点 都 不 好玩', predict: '消极' , label: '消极'
inputs: '网上 开发 女孩 的 QQ', predict: '中性' , label: '中性'
inputs: '背 你 猜 对 了', predict: '中性' , label: '中性'
inputs: '我 讨厌 你 ， 哼哼 哼 。 。', predict: '消极' , label: '消极'


In [19]:
predict("家人们咱就是说一整个无语住了 绝绝子叠buff")

inputs: '家人们咱就是说一整个无语住了 绝绝子叠buff', predict: '中性'
